In [ ]:
import sys
import subprocess
def install_packages():
    required_packages = [
        "MetaTrader5",
        "numpy",
        "pandas",
        "scikit-learn",
        "joblib",
        "pyarrow",
        "fastparquet"
    ]

    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Instalando {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    print("Todos os pacotes estão instalados.")

install_packages()